In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from itertools import chain
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import numpy as np
import re
from numpy import random


In [4]:
all_countries=pd.read_csv("C:/Users/homa/Python-Notbook/TradeMap/all_countries.csv")
Product_Code=pd.read_csv("C:/Users/homa/Python-Notbook/TradeMap/Product_Codem.csv")
all_countries.head(2)

,Country,Unnamed: 1,Country1,Unnamed: 3
0,'Afghanistan,NaN,Afghanistan,NaN
1,'Albania,NaN,Albania,NaN


In [5]:

driver= webdriver.Chrome()


In [6]:
driver.get("https://www.trademap.org/Index.aspx")

In [7]:
driver.maximize_window()

In [8]:

waits= WebDriverWait(driver, 30)

## Login Users

In [9]:
def login():
    try:
        logintext=driver.find_element(By.CSS_SELECTOR ,("#ctl00_MenuControl_Label_Login"))
        if logintext.text=='Login': 
                
            login=waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Login')))
            login.click()
            url= driver.current_url
            if url!='https://www.trademap.org/Index.aspx': 
                getemail= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Username')))
                getemail.send_keys('homa.abedinzadeh@gmail.com')
                #time.sleep(random.randint(5,20))
                getpassword= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Password')))
                getpassword.send_keys('Aa@97019701')
                #time.sleep(random.randint(5,20))
                waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.container > div > div:nth-child(2) > div > div > div > form > fieldset > div:nth-child(4) > div > button'))).click()
            
    
    except:
        print  ('LOGIN Error DAD')


## Del 


In [10]:
def delete():
    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Image_deleteProduct'))).click()
    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Image_DeleteCountry'))).click()  
 

In [9]:
print (Product_Code['ProductCode'].iloc[1])

'200490


## Get Codes and Country

In [15]:
df=pd.DataFrame()


for i in range (len(Product_Code)):
    for j in range (len (all_countries)):
        try:
            login()
            delete()

    ## Select Product 
            inputproduct= waits.until(EC.element_to_be_clickable ((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
            inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])

            clkinput= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,  '#ctl00_PageContent_RadComboBox_Product_c0'))).click()


    ## select Country
            inputContry= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,  '#ctl00_PageContent_RadComboBox_Country_Input')))
            inputContry.send_keys(all_countries['Country'].iloc[j])
            time.sleep(3)
            clkinput2= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,  '#ctl00_PageContent_RadComboBox_Country_c0'))).click()

            
            btnIndicator = driver.find_element(By.NAME ,("ctl00$PageContent$Button_TradeIndicators"))
            if btnIndicator.is_enabled():
                
                btn=waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()
                
    ##300 Pages
                incshow=waits.until(EC.element_to_be_clickable((By.XPATH , "//select[@id='ctl00_PageContent_GridViewPanelControl_DropDownList_PageSize']")))
                drop=Select(incshow)
                drop.select_by_value('300')
    ## Get Table
                all_tables = pd.read_html(driver.page_source, attrs={'id': 'ctl00_PageContent_MyGridView1'})
                temp = all_tables[0]

                if temp.shape[0] != 1:
                    temp = temp.iloc[1:, 1:].iloc[:, :-4]
                    new_header = temp.iloc[0]
                    temp = temp[1:]
                    temp.columns = new_header

                    # افزودن اطلاعات کشور و کد محصول
                    temp['Importer'] = all_countries['Country'].iloc[j]
                    temp['productcode'] = re.sub(r'[^\w]', ' ', Product_Code['ProductCode'].iloc[i])
                    temp.drop(temp[temp['Exporters'] == 'Total'].index, inplace=True)

                    # تغییر نام ستون‌ها
                    average = temp.columns[15]
                    sharein = temp.columns[3]
                    temp.rename(columns={average: 'Average tariff%', sharein: 'ShareinCounryImports'}, inplace=True)

                    # افزودن به DataFrame اصلی
                    df = pd.concat([df, temp], axis=0)
                    temp = pd.DataFrame()
                
                else:
                    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Menu_Home'))).click()
                    time.sleep(random.randint(2,5))
                    delete() 
                    j+=1  
                
                                

            else:
                    delete()
                    j+=1
            
            print(all_countries['Country'].iloc[j]+'  '+ str (j)  )    
        except Exception as e:
                print(f"Error selecting option {j}: {e}")
                site = driver.current_url
                if site!= 'https://www.trademap.org/Index.aspx':
                     driver.get('https://www.trademap.org/Index.aspx')
            
             
            
        site = driver.current_url
        if site!= 'https://www.trademap.org/Index.aspx':
                waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Menu_Home'))).click()
                time.sleep(random.randint(2,5))
                delete()
        else:
                driver.get('https://www.trademap.org/Index.aspx')  
                time.sleep(random.randint(2,5))
                delete()
        j+=1
        

i+=1

'Afghanistan  0
'Albania  1
'Algeria  2
'American Samoa  3
'Andorra  4
'Angola  5
'Anguilla  6
'Antigua and Barbuda  7
Error selecting option 8: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF62E93B125+29573]
	(No symbol) [0x00007FF62E8AFF50]
	(No symbol) [0x00007FF62E76B6EA]
	(No symbol) [0x00007FF62E7722BE]
	(No symbol) [0x00007FF62E774637]
	(No symbol) [0x00007FF62E7746F0]
	(No symbol) [0x00007FF62E7B9BC0]
	(No symbol) [0x00007FF62E7E72FA]
	(No symbol) [0x00007FF62E7B2FF6]
	(No symbol) [0x00007FF62E7E7510]
	(No symbol) [0x00007FF62E8086BC]
	(No symbol) [0x00007FF62E7E70A3]
	(No symbol) [0x00007FF62E7B12DF]
	(No symbol) [0x00007FF62E7B2441]
	GetHandleVerifier [0x00007FF62EC6C76D+3377613]
	GetHandleVerifier [0x00007FF62ECB7

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF62E93B125+29573]
	(No symbol) [0x00007FF62E8AFF50]
	(No symbol) [0x00007FF62E76B6EA]
	(No symbol) [0x00007FF62E7BF815]
	(No symbol) [0x00007FF62E7BFA6C]
	(No symbol) [0x00007FF62E80B917]
	(No symbol) [0x00007FF62E7E733F]
	(No symbol) [0x00007FF62E8086BC]
	(No symbol) [0x00007FF62E7E70A3]
	(No symbol) [0x00007FF62E7B12DF]
	(No symbol) [0x00007FF62E7B2441]
	GetHandleVerifier [0x00007FF62EC6C76D+3377613]
	GetHandleVerifier [0x00007FF62ECB7B67+3685831]
	GetHandleVerifier [0x00007FF62ECACF8B+3641835]
	GetHandleVerifier [0x00007FF62E9FB2A6+816390]
	(No symbol) [0x00007FF62E8BB25F]
	(No symbol) [0x00007FF62E8B7084]
	(No symbol) [0x00007FF62E8B7220]
	(No symbol) [0x00007FF62E8A607F]
	BaseThreadInitThunk [0x00007FFD4DF2257D+29]
	RtlUserThreadStart [0x00007FFD4EDEAF28+40]
